# Bag of words model

In [1]:
import numpy as np

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear
from torch.nn.functional import softmax, relu, tanh
from torchtext.vocab import Vectors, GloVe, CharNGram, FastText

from sklearn.manifold import TSNE

from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook, show, push_notebook
output_notebook()


Loading BokehJS ...

In [2]:
use_cuda = torch.cuda.is_available()

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

## SST

In [3]:
TEXT = data.Field(sequential=True)
LABEL = data.Field(sequential=False)

train_set, validation_set, test_set = datasets.SST.splits(TEXT,LABEL, fine_grained=False, train_subtrees=True,filter_pred=lambda ex: ex.label != 'neutral')

In [4]:
print('train.fields', train_set.fields)
print('len(train)', len(train_set))
print('vars(train[0])', vars(train_set[0]))
print()
print('Example 2', vars(train_set[17]))

train.fields {'text': <torchtext.data.field.Field object at 0x0000021DAEF99E48>, 'label': <torchtext.data.field.Field object at 0x0000021DAEF99E80>}
len(train) 98794
vars(train[0]) {'text': ['The', 'Rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'Century', "'s", 'new', '``', 'Conan', "''", 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'Arnold', 'Schwarzenegger', ',', 'Jean-Claud', 'Van', 'Damme', 'or', 'Steven', 'Segal', '.'], 'label': 'positive'}

Example 2 {'text': ['The', 'gorgeously', 'elaborate', 'continuation', 'of', '``', 'The', 'Lord', 'of', 'the', 'Rings', "''", 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'can', 'not', 'adequately', 'describe', 'co-writer\\/director', 'Peter', 'Jackson', "'s", 'expanded', 'vision', 'of', 'J.R.R.', 'Tolkien', "'s", 'Middle-earth', '.'], 'label': 'positive'}


In [48]:
# build the vocabulary
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
#TEXT.build_vocab(train_set, max_size=None, vectors=Vectors('wiki.simple.vec', url=url))
TEXT.build_vocab(train_set, vectors=[GloVe(name='840B',dim='300'),CharNGram(),FastText()])
LABEL.build_vocab(train_set)
# print vocab information
print('len(TEXT.vocab)', len(TEXT.vocab))
print('TEXT.vocab.vectors.size()', TEXT.vocab.vectors.size())

# make iterator for splits
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_set, validation_set, test_set), batch_size=3)


# print batch information
batchsst = next(iter(train_iter))
print(batchsst.text)
print(batchsst.label)

100%|██████████████████████████████████████████████████████████████████████| 2196017/2196017 [09:40<00:00, 3781.24it/s]


MemoryError: 

In [12]:
embedding_dim = TEXT.vocab.vectors.size()[1]
num_embeddings = TEXT.vocab.vectors.size()[0]
num_classes = len(LABEL.vocab.itos)
dropout_rate = 0.2

input_dim = 100

con_dim = 200

# build the BoW model
class BoWNet(nn.Module):

    def __init__(self):
        super(BoWNet, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings, embedding_dim)
        # use pretrained embeddings
        self.embeddings.weight.data.copy_(TEXT.vocab.vectors)
        
        # add hidden layers
        # YOUR CODE HERE!
        
        self.input = Linear(in_features = embedding_dim,
                             out_features = input_dim,
                             bias = False)
        
        self.l_1 = Linear(in_features=con_dim,
                           out_features=con_dim,
                           bias = False)
        self.l_2 = Linear(in_features=con_dim,
                           out_features=con_dim,
                           bias=False)
        self.l_3 = Linear(in_features=con_dim,
                           out_features = con_dim,
                           bias = False)
        
        self.drop = nn.Dropout(p = dropout_rate)
        
        # output layer
        self.l_out = Linear(in_features=con_dim,
                            out_features=num_classes,
                            bias=False)
        
    def forward(self, x, y):
        out = {}
        # get embeddings
        x = self.embeddings(x) # (bs,len,300)
        y = self.embeddings(y)
        
        x = self.drop(x)
        y = self.drop(y)
        
        #print("X " + str(x.size()))
        #print("Y " + str(y.size()))
        
        #sum_x =  # (bs,300) 
        
        #print(x.size())
        
        #print(x.size())
        
        sum_x = torch.sum(x,0)
        sum_y = torch.sum(y,0)
        
        sum_x = sum_x.view(1,x.size()[1],embedding_dim)
        sum_y = sum_x.view(1,y.size()[1],embedding_dim)
        
        #print(sum_x.size())
        
        
        #tanh # (bs,100)
        
        sum_x = torch.tanh(self.input(sum_x))
        sum_y = torch.tanh(self.input(sum_y))
        
        
        
       # print(sum_x.size())

    
        z = torch.cat((sum_x,sum_y),2)
        
        
        z = torch.squeeze(z,0)
       # print(z.size())
        
        z = torch.tanh(self.l_1(z))     
        z = torch.tanh(self.l_2(z))
        z = torch.tanh(self.l_3(z))
       # print(z.size())
        
        z = self.drop(z)

        # Softmax
        out['out'] = self.l_out(z)
        return out

net = BoWNet()
if use_cuda:
    net.cuda()
print(net)

BoWNet(
  (embeddings): Embedding(18005, 300)
  (input): Linear(in_features=300, out_features=100, bias=False)
  (l_1): Linear(in_features=200, out_features=200, bias=False)
  (l_2): Linear(in_features=200, out_features=200, bias=False)
  (l_3): Linear(in_features=200, out_features=200, bias=False)
  (drop): Dropout(p=0.2)
  (l_out): Linear(in_features=200, out_features=3, bias=False)
)


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(net.parameters(), lr=0.001,weight_decay=0.001)

def accuracy(ys, ts):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = torch.eq(torch.max(ys, 1)[1], ts)
    # averaging the one-hot encoded vector
    return torch.mean(correct_prediction.float())

In [14]:
def construct_sentences(batch):
    """    
    Parameters
    ----------
    batch: torchtext.data.batch.Batch
    
    Returns
    -------
    [str]
    """
    return [" ".join([TEXT.vocab.itos[elm] 
                      for elm in get_numpy(batch.text[:,i])])
            for i in range(batch.text.size()[1])]

def get_labels(batch):
    """
    Parameters
    ----------
    batch: torchtext.data.batch.Batch
    
    Returns
    -------
    [str]
    """
    return [LABEL.vocab.itos[get_numpy(batch.label[i])] for i in range(len(batch.label))]


In [15]:
#derp = next(iter(val_iter))
#print(derp.text[0])

## TRAINING BOW WITH SST

In [16]:
max_iter = 10000
eval_every = 1000
log_every = 200

# will be updated while iterating
#tsne_plot = show(p, notebook_handle=True)

train_loss, train_accs = [], []

net.train()
for i, batch in enumerate(train_iter):
    if i % eval_every == 0:
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0
        val_meta = {'label_idx': [], 'sentences': [], 'labels': []}
        for val_batch in val_iter:
            output = net(val_batch.text,val_batch.text)
            # batches sizes might vary, which is why we cannot just mean the batch's loss
            # we multiply the loss and accuracies with the batch's size,
            # to later divide by the total size
           # print(val_batch.label)
            val_losses += criterion(output['out'], val_batch.label) * val_batch.batch_size
            val_accs += accuracy(output['out'], val_batch.label) * val_batch.batch_size
            val_lengths += val_batch.batch_size
            
            for key, _val in output.items():
                if key not in val_meta:
                    val_meta[key] = []
                val_meta[key].append(get_numpy(_val)) 
            val_meta['label_idx'].append(get_numpy(val_batch.label))
            val_meta['sentences'].append(construct_sentences(val_batch))
            val_meta['labels'].append(get_labels(val_batch))
        
        for key, _val in val_meta.items():
            val_meta[key] = np.concatenate(_val)
        
        # divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        
        print("valid, it: {} loss: {:.2f} accs: {:.2f}\n".format(i, get_numpy(val_losses), get_numpy(val_accs)))
        #update_plot(val_meta, 'bow', tsne_plot)
        
        net.train()
    
    output = net(batchsst.text,batchsst.text)
    batch_loss = criterion(output['out'], batchsst.label)
    
    train_loss.append(get_numpy(batch_loss))
    train_accs.append(get_numpy(accuracy(output['out'], batchsst.label)))
    
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    
    if i % log_every == 0:        
        print("train, it: {} loss: {:.2f} accs: {:.2f}".format(i, 
                                                               np.mean(train_loss), 
                                                               np.mean(train_accs)))
        # reset
        train_loss, train_accs = [], []
        
    if max_iter < i:
        break

C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


valid, it: 0 loss: 1.08 accs: 0.49

train, it: 0 loss: 1.10 accs: 0.33
train, it: 200 loss: 1.08 accs: 0.43
train, it: 400 loss: 1.05 accs: 0.50
train, it: 600 loss: 1.02 accs: 0.54
train, it: 800 loss: 1.00 accs: 0.49
valid, it: 1000 loss: 0.87 accs: 0.57

train, it: 1000 loss: 0.97 accs: 0.57
train, it: 1200 loss: 0.94 accs: 0.57
train, it: 1400 loss: 0.92 accs: 0.58
train, it: 1600 loss: 0.90 accs: 0.58
train, it: 1800 loss: 0.87 accs: 0.57
valid, it: 2000 loss: 0.76 accs: 0.52

train, it: 2000 loss: 0.86 accs: 0.57
train, it: 2200 loss: 0.84 accs: 0.60
train, it: 2400 loss: 0.81 accs: 0.57
train, it: 2600 loss: 0.81 accs: 0.55
train, it: 2800 loss: 0.80 accs: 0.55
valid, it: 3000 loss: 0.72 accs: 0.51

train, it: 3000 loss: 0.79 accs: 0.57
train, it: 3200 loss: 0.78 accs: 0.57
train, it: 3400 loss: 0.77 accs: 0.56
train, it: 3600 loss: 0.75 accs: 0.59
train, it: 3800 loss: 0.75 accs: 0.56
valid, it: 4000 loss: 0.70 accs: 0.51

train, it: 4000 loss: 0.74 accs: 0.58
train, it: 4200 l

## SNLI

In [37]:
print("Run test on SNLI...")
TEXT = datasets.snli.ParsedTextField()
LABEL = data.LabelField()
TREE = datasets.snli.ShiftReduceField()

train, val, test = datasets.SNLI.splits(TEXT, LABEL, TREE)

print("Fields:", train.fields)
print("Number of examples:\n", len(train))
print("First Example instance:\n", vars(train[0]))

TEXT.build_vocab(train)
LABEL.build_vocab(train)


train_iter, val_iter, test_iter = data.Iterator.splits((train, val, test), batch_size=3)

batch = next(iter(train_iter))
print("Numericalize premises:\n", batch.premise)
print("Numericalize hypotheses:\n", batch.hypothesis)
print("Entailment labels:\n", batch.label)

print("Test iters function")
train_iter, val_iter, test_iter = datasets.SNLI.iters(batch_size=3, trees=True)

batch = next(iter(train_iter))
print("Numericalize premises:\n", batch.premise)
print("Numericalize hypotheses:\n", batch.hypothesis)
print("Entailment labels:\n", batch.label)

#val_iter_set = next(iter(val_iter))


Run test on SNLI...
Fields: {'premise': <torchtext.datasets.snli.ParsedTextField object at 0x0000021E8EE96080>, 'premise_transitions': <torchtext.datasets.snli.ShiftReduceField object at 0x0000021E8EE960F0>, 'hypothesis': <torchtext.datasets.snli.ParsedTextField object at 0x0000021E8EE96080>, 'hypothesis_transitions': <torchtext.datasets.snli.ShiftReduceField object at 0x0000021E8EE960F0>, 'label': <torchtext.data.field.LabelField object at 0x0000021E8EE96EB8>}
Number of examples:
 549367
First Example instance:
 {'premise': ['A', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.'], 'premise_transitions': ['shift', 'shift', 'reduce', 'shift', 'shift', 'shift', 'reduce', 'reduce', 'reduce', 'shift', 'shift', 'shift', 'shift', 'shift', 'shift', 'reduce', 'reduce', 'reduce', 'reduce', 'reduce', 'shift', 'reduce', 'reduce'], 'hypothesis': ['A', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.'], 'hypothesis_transitions': ['shift',

In [46]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)
print(len(TEXT.vocab))
print(len(LABEL.vocab))
print(vars(TEXT.vocab))
print(vars(LABEL.vocab))

42392
3


{'freqs': Counter({'.': 962657, 'a': 922134, 'A': 516752, 'in': 405758, 'is': 373427, 'the': 369837, 'man': 256335, 'on': 235136, 'and': 206122, 'are': 199006, 'of': 192044, 'with': 169010, 'The': 164281, 'woman': 133171, ',': 114347, 'to': 113813, 'at': 97994, 'people': 95936, 'Two': 90406, 'wearing': 80948, 'his': 72324, 'playing': 59443, 'girl': 57818, 'boy': 56849, 'young': 56527, 'shirt': 56477, 'white': 56455, 'while': 55849, 'men': 55790, 'an': 55591, 'black': 54175, 'sitting': 53609, 'dog': 53160, 'blue': 49041, 'standing': 46291, 'her': 45539, 'red': 43062, 'for': 42178, 'group': 41852, 'down': 38962, 'walking': 38753, 'outside': 38591, 'street': 37809, 'front': 34886, 'person': 34669, 'holding': 33984, 'child': 32027, 'women': 31695, 'by': 31672, 'two': 31633, 'water': 29589, 'their': 27775, 'There': 27093, 'up': 26655, 'looking': 26295, 'one': 26193, 'People': 25193, 'An': 24601, 'as': 24586, 'Three': 23485, 'green': 23033, 'from': 22795, 'children': 22396, 'other': 21590, '

{'freqs': Counter({'entailment': 183416, 'contradiction': 183187, 'neutral': 182764}), 'itos': ['entailment', 'contradiction', 'neutral'], 'stoi': defaultdict(<function _default_unk_index at 0x0000021DABC3E048>, {'entailment': 0, 'contradiction': 1, 'neutral': 2}), 'vectors': None}


In [228]:
val_iter_set = next(iter(val_iter))


C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


In [34]:
#print(sumP, sumH)

In [35]:
#print(TEXT.build_vocab.vectors.size())

In [36]:
#embedding_dim = TEXT.vocab.vectors.size()[1]
#num_embeddings = TEXT.vocab.vectors.size()[0]
#num_classes = len(LABEL.vocab.itos)

In [33]:
max_iter = 1000
eval_every = 1000
log_every = 200

# will be updated while iterating
#tsne_plot = show(p, notebook_handle=True)

train_loss, train_accs = [], []

net.train()
for i, batch in enumerate(train_iter):
    if i % eval_every == 0:
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0
       # val_meta = {'label_idx': [], 'sentences': [], 'labels': []}
        for val_batch in val_iter:
            output = net(val_batch.premise,val_batch.hypothesis)
            # batches sizes might vary, which is why we cannot just mean the batch's loss
            # we multiply the loss and accuracies with the batch's size,
            # to later divide by the total size
            val_losses += criterion(output['out'], val_batch.label) * val_batch.batch_size
            val_accs += accuracy(output['out'], val_batch.label) * val_batch.batch_size
            val_lengths += val_batch.batch_size
            
        
        # divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        
        print("valid, it: {} loss: {:.2f} accs: {:.2f}\n".format(i, get_numpy(val_losses), get_numpy(val_accs)))
        #update_plot(val_meta, 'bow', tsne_plot)
        
        net.train()
    
    output = net(batch.premise,batch.hypothesis)
    batch_loss = criterion(output['out'], batch.label)
    
    train_loss.append(get_numpy(batch_loss))
    train_accs.append(get_numpy(accuracy(output['out'], batch.label)))
    
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    
    if i % log_every == 0:        
        print("train, it: {} loss: {:.2f} accs: {:.2f}".format(i, 
                                                               np.mean(train_loss), 
                                                               np.mean(train_accs)))
        # reset
        train_loss, train_accs = [], []
        
    if max_iter < i:
        break

C:\Users\Frederik\AppData\Roaming\Python\Python36\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


valid, it: 0 loss: 0.93 accs: 0.33

train, it: 0 loss: 0.78 accs: 0.00
train, it: 200 loss: 0.95 accs: 0.34
train, it: 400 loss: 0.98 accs: 0.32
train, it: 600 loss: 0.98 accs: 0.32
train, it: 800 loss: 0.96 accs: 0.34
valid, it: 1000 loss: 0.94 accs: 0.33

train, it: 1000 loss: 0.98 accs: 0.32
